api call

In [ ]:
import random
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-mE91TMZY8yikxpif8fBa64F0BaBa4d76BcCdD0Cb13F437D2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content.strip('\n')
    # return response


logs = '''`Remoting started; listening on addresses :[akka.tcp://sparkExecutorActorSystem@mesos-slave-07:55904]`'''
system_prompt = '''here are some types of variables, descriptions and examples in log message:
<OID>: Identification information of an object. 
Added attempt `1445144423722` to list of failed maps.
<LOI>: Location information of an object. 
Adding path spec: `/mapreduce`.
<OBN>: Name of an object. 
ServerFileSystem domain `root10-local` is full.
<TID>: Type information of an object or an action. 
Using configuration type `1`.
<SID>: Status of a switch variable. 
Saw change in network reachability (isReachable= `2`).
<TDA>: Time or duration of an action. 
Scheduled snapshot period at `10` second(s).
<CRS>: Information of computing resource. 
Combo kernel: `126MB` LOWMEM available.
<OBA>: Amount of an object. 
Total of `23` ddr error(s) detected and corrected.
<STC>: Status code of an object or an action. 
mod-jk child workerEnv in error state `7`.
<OTP>: Other information does not belong to the above categories. 
calvisitor kernel: payload Data `0700` to list of failed maps. You will be provided with a log message delimited by backticks.. You must abstract variables with `{{type}}` to extract the corresponding template.
Print the input log's template delimited by backticks.
log message: `Remoting started; listening on addresses :[akka.tcp://sparkExecutorActorSystem@mesos-slave-07:55904]`'''

messages = [
    {'role': 'user', 'content': system_prompt},
]
# 
print(get_responce(messages))

In [ ]:
from parsing_copy import get_candidate_template
logs = ['connection from 210.245.165.136 () at Wed Jun 22 13:16:30 2005']
templates = ['connection from <*> (<*>) at <*>', 'Kerberos authentication failed',
             'input_userauth_request: invalid user  [preauth]']
print(get_candidate_template(templates, logs))

In [ ]:
import re

log = "connection from 203.101.45.59 () at Sun Jul  3 10:05:25 2005"
template = "connection from <*> () at <*>"

# 将模板中的"<*>"替换为正则表达式的通配符".*?"
template = template.replace("<*>", "(.*?)")

# 使用正则表达式匹配日志字符串
match = re.search(template, log)

# 提取匹配的字符串
if match:
    print(match.groups())

Evaluate_gpt

In [7]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac' # logpub
datasets = table_order.split(' ')
m,n,p,q = [],[],[],[]
for dataset in datasets:
    file = f'outputs/parser/logpub_0shot/{dataset}.csv'  # Fifth_=_0.1
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a,b,c,d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print('avg---------: group Accuracy: %.4f, Message-Level Accuracy: %.4f, Edit Distance: %.4f' % (sum(m)/len(m), sum(n)/len(n), sum(p)/len(p)))

# 81.0 71.2

        HDFS: group Accuracy: 1.0000, Message-Level Accuracy: 0.7950, Edit Distance: 1.1800
      Hadoop: group Accuracy: 0.9855, Message-Level Accuracy: 0.6070, Edit Distance: 20.2145
       Spark: group Accuracy: 0.9220, Message-Level Accuracy: 0.8790, Edit Distance: 2.0495
   Zookeeper: group Accuracy: 0.9945, Message-Level Accuracy: 0.9655, Edit Distance: 0.3505
         BGL: group Accuracy: 0.9750, Message-Level Accuracy: 0.9520, Edit Distance: 1.0655
         HPC: group Accuracy: 0.8355, Message-Level Accuracy: 0.7605, Edit Distance: 2.7165
 Thunderbird: group Accuracy: 0.9820, Message-Level Accuracy: 0.6285, Edit Distance: 4.1245
       Linux: group Accuracy: 0.3530, Message-Level Accuracy: 0.4910, Edit Distance: 5.0515
   HealthApp: group Accuracy: 0.7395, Message-Level Accuracy: 0.5360, Edit Distance: 7.4655
      Apache: group Accuracy: 1.0000, Message-Level Accuracy: 0.9780, Edit Distance: 0.2300
   Proxifier: group Accuracy: 1.0000, Message-Level Accuracy: 0.0180, Edit Dist

In [ ]:
from evaluator import evaluate
evaluate('outputs/parser/0125_1shot_brandnew/Linux.csv', 'Linux')

In [ ]:
# discard the target dataset
dataset = 'Apache'
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
            'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
datasets.remove(dataset)
global demonstration_templates
global demonstration_logs
demonstration_templates = []
demonstration_logs = []
for d in datasets:
    df = pd.read_csv(f'dataset\{d}\{d}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in demonstration_templates:
            demonstration_templates.append(template)
            demonstration_logs.append(log)


In [ ]:
import difflib


def find_closest_matches(target, list1, list2, n=5):
    matches = difflib.get_close_matches(target, list1, n=n,cutoff=0.1)
    indices = [list1.index(match) for match in matches]
    return [list1[index] for index in indices],[list2[index] for index in indices]

list1, list2 = find_closest_matches('PacketResponder 1 for block blk_38865049064139660 terminating', demonstration_templates, demonstration_logs)

print(list1)
print(list2)

In [ ]:
import re


def replace_numbers_with_zero(text):
    return re.sub(r'\d+(\.\d+)?', '0', text)


input = 'Failed password for root from 5.36.59.76 port 42393 ssh2'
print(replace_numbers_with_zero(input))
print(replace_numbers_with_zero(replace_numbers_with_zero(input)))

In [ ]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# datasets = ['OpenSSH']
m, n, p, q = [], [], [], []
for dataset in datasets:
    file = f'outputs/parser/Fifth_=_0.1/{dataset}.csv'  # Fourth_guding
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a, b, c, d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print(sum(m)/len(m))
print(sum(n)/len(n))
print(sum(p)/len(p))

# 81.0 71.2

In [ ]:
dataset = 'Linux'
df = pd.read_csv(
    f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
list = df['EventTemplate'].tolist()
templates = []
for item in list:
    if item not in templates:
        templates.append(item)
print(len(templates))
for template in templates:
    print(template)


验证按' '分词有多少词，以及含数字的词占多少

In [ ]:
import re
import pandas as pd
from post_process import correct_single_template
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Android','Hadoop', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# datasets = ['Linux']


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words


a = 0
b = 0
list = []
list2 = []
k = 0
temp = ['structured', 'templates', 'Content', 'EventTemplate']
tokens = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    print('-' * 20)
    print(dataset)

    # 验证长度为1-3的tempalte是否含有<*>
    # NOTE: 长度为1时，不用解析，2-3仍然需要解析
    # DS
    # NOTE: 没有template中含有'  '
    # BL+US
    # NOTE: boolen变量主要出现在Android中 null:Mac, Android root:many datasets admin:OpenSSH, Thunderbird
    # BL = ['true', 'false']
    # US = ['null', 'root', 'admin']
    # DG
    # NOTE: 没有template含有纯数字
    #
    tem = []
    for log, template in zip(logs, templates):
        if '/' in log and template not in tem:
            words = tokenize(log)
            tem.append(template)
            print(words)
            print(template)
            print('-' * 20)
            
            # print('-' * 20)
    # 验证按' '分词有多少词，以及含数字的词占多少
    # list2.append(len(list))    
    # total_words = 0
    # words_with_numbers = 0

    # for s in list:
    #     words = s.split()
    #     total_words += len(words)
    #     words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    # a+=total_words
    # b+=words_with_numbers


# print(f"{b} / {a}")

GPT-4 call

In [ ]:
from openai import OpenAI
import httpx
client = OpenAI(
    base_url="https://4.0.996444.icu/v1", 
    api_key="sk-1XhqpmIwxu9nCuCD51FcA00948F54eDf9cA4765998Ce8f0d",
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)
message_list = [
     {"role": "user", "content": "Hello"}, 
    ]

response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=message_list,
    temperature=0,
    max_tokens=1024
)

print(response)
response = response.choices[0].message.content

In [ ]:

import re
tokenize_pattern = r'[#= ,]'
log_content = 'xy=123,z=789,7#8#9 sex you '
words = re.split(tokenize_pattern, log_content)
print(words)

In [ ]:
import re


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words


# root, admin, 

logs = []
logs.append(
    'proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01')
logs.append(
    'proxy.cse.cuhk.edu.hk:5070 close, 451 bytes sent, 18846 bytes (18.4 KB) received, lifetime <1 sec')
logs.append('proxy.cse.cuhk.edu.hk:5070 close, 1165 bytes (1.13 KB) sent, 3098 bytes (3.02 KB) received, lifetime 00:01')

for log in logs:

    print(tokenize(log))

Drain

In [ ]:
from drain3 import TemplateMiner

# 创建一个模板挖掘器实例
miner = TemplateMiner()


# 添加一些日志行
logs = list(set(prompt.split('\n')))
for log in logs:
    result = miner.add_log_message(log)
    if result is not None:
        print(f"New template: {result}")

# 打印所有的日志模板
print("Log templates:")
for template in miner.drain.clusters:
    print(template.get_template())

In [ ]:
%pip install Levenshtein

In [ ]:
from Levenshtein import distance


def closest_template(target, templates):
    min_distance = float('inf')
    closest_template = None
    for template in templates:
        d = distance(target, template)
        if d < min_distance:
            min_distance = d
            closest_template = template
    return closest_template

templates = [
    '<*> close, <*> bytes sent, <*> bytes received, lifetime <1',
    'proxy.cse.cuhk.edu.hk:<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1',
    '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>',
    '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>',
]

print(closest_template('proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01', templates))

生成候选的labelled logs

In [ ]:
import json
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

data = {}
templates = []
logs = []
for dataset in datasets:
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in templates:
            templates.append(template)
            logs.append(log)
    # 'logs': logs_c, 'templates': templates_c,
    print(dataset, len(logs))


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(target, lst, n=3):
    vectors = [np.array([ord(c) for c in s]).reshape(1, -1) for s in lst]
    target_vector = np.array([ord(c) for c in target]).reshape(1, -1)
    similarities = cosine_similarity(vectors, target_vector)
    most_similar_indices = np.argsort(similarities, axis=0)[-n:].flatten()[::-1]
    return [lst[i] for i in most_similar_indices]

In [ ]:
import pandas as pd
import re
# read log
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print(dataset)
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log,template in zip(logs, templates):    
        if 'kb' in log.lower() and template not in list:
            print(log)
            list.append(template)


list = ['sec', 'KB']

查看logpub中oracle template不合理的部分

In [4]:
from post_process import correct_single_template
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print('+' * 20 + dataset)
    df = pd.read_csv(
        f'dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv')
    templates = df['EventTemplate'].tolist()
    for template in templates:
        if correct_single_template(template) != template:
            print(correct_single_template(template))
            print(template)
            print('-' * 20)

++++++++++++++++++++HDFS
++++++++++++++++++++Hadoop
maxContainerCapability: <memory:<*>, vCores:<*>
maxContainerCapability: <memory:<*>, vCores:<*>>
--------------------
++++++++++++++++++++Spark
++++++++++++++++++++Zookeeper
******* GOODBYE <*> ********
******* GOODBYE <*>:<*> ********
--------------------
Accepted socket connection from <*>
Accepted socket connection from <*>:<*>
--------------------
Cannot open channel to <*> at election address <*>
Cannot open channel to <*> at election address <*>:<*>
--------------------
Client attempting to establish new session at <*>
Client attempting to establish new session at <*>:<*>
--------------------
Client attempting to renew session <*> at <*>
Client attempting to renew session <*> at <*>:<*>
--------------------
Closed socket connection for client <*> (no session established for client)
Closed socket connection for client <*>:<*> (no session established for client)
--------------------
Closed socket connection for client <*> which ha

try to conclude the types of variables

In [ ]:
import pandas as pd 
import re

def extract_variables(log, template):
    # 将模板中的 <*> 替换为正则表达式的捕获组 (.*?)
    # 为了避免正则表达式的特殊字符导致的问题，先将模板中除了 <*> 外的其他部分进行转义
    # 然后将 <*> 替换为正则表达式的捕获组
    # 这里假设模板中的 <*> 不紧邻正则特殊字符，如果有，需要更复杂的处理
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  # 添加开始和结束锚点以确保完整匹配

    matches = re.search(regex, log)
    if matches:
        return matches.groups()
    else:
        return []

def check_variable(variable):
    variable = variable.strip()
    if variable.startswith('/'):
        return 'path'
    if variable.startswith('0x'):
        return 'address'
    if re.match(r'\b-?\d+(\.\d+)?\b|\b0x[0-9a-fA-F]+\b', variable):
        return 'number'
    if re.match( r'^[a-zA-Z]+$', variable):
        return 'word'
    if re.match(  r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b', variable):
        return 'ip'
    else:
        return 'null'

table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

dict = {}

for dataset in datasets:
    dict[dataset] = {}
    print('-' * 20)
    print(dataset + ':')
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    been_used = []
    for log, template in zip(logs, templates):
        if template not in been_used:
            been_used.append(template)
            variables = extract_variables(log, template)
            for variable in variables:
                type = check_variable(variable)
                if type != 'null':
                    if type not in dict[dataset]:
                        dict[dataset][type] = 1
                    else:
                        dict[dataset][type] += 1
                else:
                    print(variable)

print(dict)

Test the unseen accurray

In [4]:
from collections import Counter
from evaluator import evaluate
import os
import pandas as pd
from sklearn.metrics import accuracy_score

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
# table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac' # logpub
datasets = table_order.split(' ')
a = 0
b = 0
for dataset in datasets:
    file = '0125_0shot'
    input = f'outputs/parser/{file}/{dataset}.csv'  
    output = f'outputs/unseen/{file}/{dataset}.csv'
    os.makedirs(f'outputs/unseen/{file}', exist_ok=True)
    df = pd.read_csv(input)
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    freq = Counter(templates)
    logs_after = []
    templates_after = []
    unseen_templates = [item for item, count in freq.items() if count == 1]
    for log, template in zip(logs, templates):
        if template in unseen_templates:
            logs_after.append(log)
            templates_after.append(template)
    accuracy_exact_string_matching = accuracy_score(templates_after, logs_after, normalize=False)
    length = len(logs_after)
    a += length
    b += accuracy_exact_string_matching
    dataset = ' ' * (12 - len(dataset)) + dataset
    print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %(dataset, length ,accuracy_exact_string_matching/length))
    

print(b/a)

# 81.0 71.2

        HDFS: len of unseen log: 0002, Message-Level Accuracy: 0.0000
      Hadoop: len of unseen log: 0069, Message-Level Accuracy: 0.2464
       Spark: len of unseen log: 0019, Message-Level Accuracy: 0.5789
   Zookeeper: len of unseen log: 0019, Message-Level Accuracy: 0.2105
         BGL: len of unseen log: 0044, Message-Level Accuracy: 0.0909
         HPC: len of unseen log: 0010, Message-Level Accuracy: 0.5000
 Thunderbird: len of unseen log: 0095, Message-Level Accuracy: 0.3579
     Windows: len of unseen log: 0020, Message-Level Accuracy: 0.4500
       Linux: len of unseen log: 0091, Message-Level Accuracy: 0.3407
     Android: len of unseen log: 0051, Message-Level Accuracy: 0.2353
   HealthApp: len of unseen log: 0031, Message-Level Accuracy: 0.3226
      Apache: len of unseen log: 0000, Message-Level Accuracy: nan
   Proxifier: len of unseen log: 0000, Message-Level Accuracy: nan
     OpenSSH: len of unseen log: 0005, Message-Level Accuracy: 0.2000
   OpenStack: len of unsee

C:\Users\xiaoy\AppData\Local\Temp\ipykernel_12388\3483590769.py:35: RuntimeWarning: invalid value encountered in scalar divide
  print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %(dataset, length ,accuracy_exact_string_matching/length))
C:\Users\xiaoy\AppData\Local\Temp\ipykernel_12388\3483590769.py:35: RuntimeWarning: invalid value encountered in scalar divide
  print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %(dataset, length ,accuracy_exact_string_matching/length))
